In [1]:
import pyodbc, json, requests, random, lxml
from time import sleep
from io import StringIO
from random import randint
from random import randrange
from html.parser import HTMLParser
from faker import Faker
import re
from lxml.html.clean import Cleaner

In [2]:
##HOST SERVER
server = 'iproject3.icasites.nl' 
database = 'iproject3' 
username = 'iproject3' 
password = 'aeedZ8u8'

##LOCAL SERVER
#server = 'localhost' 
#database = 'school' 
#username = 'SA' 
#password = 'yourStrong(!)Password'

cleaner = Cleaner()
cleaner.javascript = True # This is True because we want to activate the javascript filter
cleaner.style = True      # This is True because we want to activate the styles & stylesheet filter
cleaner.remove_tags=('span', 'font', 'div', 'p', 'div')

In [3]:
def addRowToFile(text):
    f = open("SQL-IMPORT-Items.txt", "a")
    f.write(str(text) + "\n")
    f.close()
    return

In [4]:
def makeConnection():
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor() 
    return cursor

In [5]:
def closeConnection(cursor):
    cursor.close()
    return

In [6]:
def doSelectQuery(sql, cursor):
    cursor.execute(sql)
    return cursor.fetchall()

In [7]:
def doInsertQuery(sql, cursor):
    cursor.execute(sql)
    cursor.commit()
    return

In [8]:
def doInsertQueryAndGetID(sql, cursor):
    cursor.execute(sql)
    ID = cursor.fetchone()[0]
    cursor.commit()
    return ID

In [9]:
def registerAsSellerByAccountID(ID, cursor):

    mastercardPrefixList = [['5', '1'], ['5', '2'], ['5', '3'], ['5', '4'], ['5', '5']]
    DutchBanken = ["ABN Amro", "ASN Bank", "Bunq", "DHB Bank", "ING Bank", "Knab", "Rabobank", "SNS Bank", "Triodos Bank"]
    BillingNumbers = ["NL59INGB3034565194","NL91INGB6971032004","NL13INGB2327776783","NL24ABNA8502137913","NL79ABNA6502761380","NL95RABO2560035731","NL06ABNA8009649546","NL51ABNA7103072574","NL92RABO7939901389","NL83INGB2786219639","NL35ABNA9732254661","NL16RABO8611765885","NL36RABO6008851617","NL12RABO2952671982","NL28INGB6291533782","NL63INGB3779588560","NL46RABO7116259503","NL63RABO4760808973","NL51ABNA5993159871","NL98ABNA1438227787","NL68RABO3031628489","NL22INGB9565235778","NL82ABNA2187209016","NL93RABO5040995164","NL26INGB4871952010","NL89ABNA3034565194","NL41INGB8683750361","NL89INGB6034837898","NL13ABNA7569006453","NL89ABNA9906008334","NL36ABNA8694866803","NL65RABO7440639480","NL73INGB6143727961","NL82INGB4197809344","NL71RABO7659536168","NL98ABNA9442141368","NL66RABO6971032004","NL58RABO6583086565","NL27INGB5647918042","NL62INGB2067756052","NL32ABNA3146454459","NL18RABO8253182457","NL96INGB8720446314","NL89ABNA3309755649","NL40ABNA9859479577","NL66INGB9224568173","NL76RABO8675663943","NL34INGB4520711568"]
    Bankname = random.choice(DutchBanken)
    BankAccountNumber = random.choice(BillingNumbers)
    ControlOptionName = "Creditcard"
    CreditcardNumber = generateCard("mastercard")

    sql = "INSERT INTO Seller (AccountID, Bankname, BankAccountNumber, CreditcardNumber, ControlOptionName) VALUES ('"+str(ID)+"', '"+str(Bankname)+"', '"+str(BankAccountNumber)+"', '"+str(CreditcardNumber)+"', '"+str(ControlOptionName)+"')"
    
    f = open("SQL-IMPORT-SELLER.txt", "a")
    f.write(sql + "\n")
    f.close()
    return


In [10]:
def genereteRandomAuctionDates():
    fake = Faker()
    
    begindate = fake.date_time_between(start_date='-'+ str(random.randint(0, 20)) +'d', end_date='now')
    enddate = fake.date_time_between(start_date=begindate, end_date='+'+ str(random.randint(0, 80)) +'d')

    return begindate.strftime("%Y-%m-%d  %H:%M:%S.000"), enddate.strftime("%Y-%m-%d  %H:%M:%S.000")

In [11]:
def generateCard(type):
    """
    Prefill some values based on the card type
    """
    card_types = ["americanexpress","visa13", "visa16","mastercard","discover"]

    def prefill(t):
        # typical number of digits in credit card
        def_length = 16

        """
        Prefill with initial numbers and return it including the total number of digits
        remaining to fill
        """
        if t == card_types[0]:
            # american express starts with 3 and is 15 digits long
            # override the def lengths
            return [3, randint(4,7)], 13

        elif t == card_types[1] or t == card_types[2]:
            # visa starts with 4
            if t.endswith("16"):
                return [4], def_length - 1
            else:
                return [4], 12

        elif t == card_types[3]:
            # master card start with 5 and is 16 digits long
            return [5, randint(1,5)], def_length - 2

        elif t == card_types[4]:
            # discover card starts with 6011 and is 16 digits long
            return [6, 0, 1, 1], def_length - 4

        else:
            # this section probably not even needed here
            return [], def_length

    def finalize(nums):
        """
        Make the current generated list pass the Luhn check by checking and adding
        the last digit appropriately bia calculating the check sum
        """
        check_sum = 0

        #is_even = True if (len(nums) + 1 % 2) == 0 else False

        """
        Reason for this check offset is to figure out whether the final list is going
        to be even or odd which will affect calculating the check_sum.
        This is mainly also to avoid reversing the list back and forth which is specified
        on the Luhn algorithm.
        """
        check_offset = (len(nums) + 1) % 2

        for i, n in enumerate(nums):
            if (i + check_offset) % 2 == 0:
                n_ = n*2
                check_sum += n_ -9 if n_ > 9 else n_
            else:
                check_sum += n
        return nums + [10 - (check_sum % 10) ]

    # main body
    t = type.lower()
    if t not in card_types:
        return

    initial, rem = prefill(t)
    so_far = initial + [randint(1,9) for x in range(rem - 1)]
    #print ("".join(map(str,finalize(so_far))))
    return "".join(map(str,finalize(so_far)))


In [12]:
def searchForConditionOrCreate(condition, cursor):
    
    sql = "SELECT * FROM Condition WHERE Name = '" + str(condition).replace(" ", "") + "'"
    result = doSelectQuery(sql, cursor)
    
    if(result):
        return result[0][0]
    else:
        sql = "INSERT INTO Condition (Name) OUTPUT INSERTED.ID VALUES ('" + str(condition).replace(" ", "") + "')"
        result = doInsertQueryAndGetID(sql, cursor)
        return result

In [13]:
def getSellerIDByUsername(username, cursor, registerSeller):
    sql = "SELECT * FROM Account WHERE Username = '" + str(username.replace(" ", "")) + "'"
    result = doSelectQuery(sql, cursor)
    if(registerSeller == True):
        registerAsSellerByAccountID(result[0][0], cursor)
    return result[0][0]

In [14]:
def getCountryIDByCode(code, cursor):
    sql = "SELECT * FROM Country WHERE CountryCode = '" + str(code.replace(" ", "")) + "'"
    result = doSelectQuery(sql, cursor)
    return result[0][0]

In [15]:
def getCategoriesAndItemID(cursor):
    SQL = "SELECT ID, CategoryID FROM Item"
    result = doSelectQuery(SQL, cursor)
    return result

In [16]:
def transferItemDataColumnsAndCreateFile(row, cursor):
    
    paymentMethodsArray = ["Contant", "Cash", "Bank", "Giro", "Tikkie", "Bank/Giro", "Contant, Bank, Giro"]
    paymentInstructionsArray = ["Overschrijving moet ontvangen zijn binnen 10 dagen na verkoop.", "Het liefst alleen contant betalebn!", "Bied maar een leuk bedrag en dan mag je hem ophalen volgende de betaal methoden. Bij Contant kun je hem afhalen", "Payment only through Bank", "Only cash payment are allowed"]
    shippingCostsArray = ["1.95","2.95","3.95","4.95","5.95","6.95","7.95","8.95","9.95","10.95","11.95","12.95", "0"]
    sendInstructionsArray = ["Liever alleen verzenden vanwege COVID-19", "Liever alleen verzenden vanwege Corona Virus", "Graag verzenden omdat ik van me dozen af moet", "Dit artikel vind ik te groot om te verzenden dus haal hem maar op", "I think this article is too big to ship, so pick it up","Prefer to ship only because of COVID-19", "Prefer to ship only because of Corona Virus", "Please ship because I have to get rid of boxes", ""]
    
    GeneratedDates = genereteRandomAuctionDates()
    
    
    ID = row[0]
    Title = row[1].replace("'","")
    Description = descriptionCleaner(row[2])
    City = row[5].replace(" ", "").replace("'","")
    CountryID = getCountryIDByCode(row[6], cursor)
    StartingPrice = row[8].replace(" ", "")
    StartDate = GeneratedDates[0]
    EndDate = GeneratedDates[1]
    PaymentMethod = random.choice(paymentMethodsArray)
    PaymentInstructions = random.choice(paymentInstructionsArray)
    ShippingCosts = random.choice(shippingCostsArray)
    SendInstructions = random.choice(sendInstructionsArray)
    SellerID = getSellerIDByUsername(row[7], cursor, True)
    ItemCategoryID = row[3]
    Thumbnail = row[11]
    ConditionID = searchForConditionOrCreate(row[10], cursor)
    Currency = row[9]
    
    SQL = "INSERT INTO Item (ID, Title, City, CountryID, StartingPrice, StartDate, EndDate, PaymentMethod, PaymentInstructions, ShippingCosts, SendInstructions, SellerID, CategoryID, Thumbnail, ConditionID, Currency, Description) VALUES (" + str(ID) + ", '" + str(Title) + "','" + str(City) + "'," + str(CountryID) + ",'" + str(StartingPrice) + "','" + str(StartDate) + "','" + str(EndDate) + "','" + str(PaymentMethod) + "','" + str(PaymentInstructions) + "','" + str(ShippingCosts) + "','" + str(SendInstructions) + "'," + str(SellerID) + "," + str(ItemCategoryID) + ",'" + str(Thumbnail) + "'," + str(ConditionID) + ",'" + str(Currency) + "','" + str(Description) + "')"
    addRowToFile(SQL)
    #doInsertQuery(SQL, cursor)

    return 

In [17]:
def transferFileDataColumnsAndCreateFile(row, cursor):
    ItemID = row[0]
    FileName = row[1]
    
    SQL = f"INSERT INTO [File] (Name, ItemID) VALUES ('/pics/{FileName}',{ItemID})"
    
    
    f = open("SQL-IMPORT-Files.txt", "a")
    f.write(SQL + "\n")
    f.close()
    #doInsertQuery(SQL, cursor)
    return

In [23]:
def transferCategoryDataColumnsAndCreateFile(row, cursor):
    
    ItemID = row[0]
    CategoryID = row[1]
    
    SQL = f"INSERT INTO CategoriesInItem (ItemID, CategoryID) VALUES ({ItemID},{CategoryID})"
    
    
    f = open("SQL-IMPORT-CAT.txt", "a")
    f.write(SQL + "\n")
    f.close()
    
    return

In [19]:
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [20]:
def descriptionCleaner(description):
    re_script = re.compile('<\s*script[^>]*>.*?<\s*/\s*script\s*>', re.S | re.I)
    css_script = re.compile('<\s*style[^>]*>.*?<\s*/\s*style\s*>', re.S | re.I)
    description = " ".join(description.split())
    description = description.rstrip()
    description = strip_tags(description)
    try:
        description = cleaner.clean_html(description)
    except: 
        pass
    
    description = re_script.sub('',description)
    description = css_script.sub('',description)
    
    description = re.sub(' +', ' ', description)
    description = description.replace('\n',' ')
    description = description.replace('/n',' ')
    description = description.replace("'","")
    description = description.replace("amp;", "")
    description = description.replace('\t',' ')
    description = description.replace("   ", " ")
    description = description.replace("<div>", "")
    description = description.replace("</div>", "")
    description = description.replace("Your browser does not support JavaScript. To view this page, enable JavaScript if it is disabled or upgrade your browser.", "")
    #print(description)
    return description
    

In [21]:
def main():
    cursor = makeConnection()
    categories = getCategoriesAndItemID(cursor)
    
    print("Starting...")
    for category in categories:
        transferCategoryDataColumnsAndCreateFile(category, cursor)
    
    #print("Starting with Items...")
    #count = 0 
    #doInsertQuery("SET IDENTITY_INSERT Item ON", cursor)
    #for item in items:
    #    transferItemDataColumnsAndCreateFile(item, cursor)    
    #print("Items and Sellers in TXT!")
    
    #print("Starting with Files...")
    #files = doSelectQuery("SELECT * FROM Illustraties_OLD",cursor)  
    #for file in files:
    #    transferFileDataColumnsAndCreateFile(file, cursor)
    #print("Files in TXT!")
    
    closeConnection(cursor)
    print("Klaar!")
        

In [24]:
if __name__ == "__main__":
    main()

Starting...
Klaar!
